# Неделя 4. Вторник
## Обучение без учителя
### Кластеризация текстов

In [270]:
import pandas as pd
import numpy as np
import re
import pymorphy3

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, rand_score

✏️  Сегодняшним вашим заданием будет кластеразиция текстов. 

* У нас имеется выборка из 5000 сообщений из новостных телеграм каналов на разные тематики
* Необходимо предобработать данные, выделить кластеры
* Собирать будем такой папйлайн `предварительный препроцессинг` __->__ `удаление стоп-слов` __->__ `лемматизация` __->__ `векторизация текста` __->__ `понижение размерности` __->__ `кластеризация`

__1.__ Подгрузите датасет `aux/tg_channels.csv`

In [3]:
# your_code
df = pd.read_csv("aux/tg_channels.csv")
df, label = df.drop('label', axis=1), df['label']
df.loc[110, 'content']

'РАР выбрала новых академиков 🦉Ими стали Алексей Поповичев, исполнительный директор Ассоциации «РусБренд», Роман Фирайнер, креативный директор агентства Instinct, Ольга Барская, вице-президент по развитию бизнеса OMD OM Group.Академики выбрали событием года проект «Вкусно — и точка», а его лидером — вице-президента по маркетингу Дарью Назаркину.С 2000 года Российская академия рекламы объединяет ведущих специалистов рекламного отечественного рынка. Членами академии становятся признанные эксперты отрасли, которые вносят значительный вклад в развитие коммуникационной индустрии.РАР содействует в формировании фундаментальных основ рекламной деятельности в стране, разрабатывает индустриальные стандарты и критерии оценки качества рекламного продукта, поддерживает молодых профессионалов и занимается развитием науки о коммуникациях в сфере рекламы.Подробнее: http://bit.ly/3XABExu#АКАР #РАР'

In [119]:
unique_labels, category_sizes = np.unique(label, return_counts=True)
true_k = unique_labels.shape[0]
label.unique()

array(['искусство', 'маркетинг', 'технологии', 'здоровье_медицина',
       'образование_познавательное'], dtype=object)

__2.__ Сделай необходимую предобработку текста `aux/tg_channels.csv`. Всю ее оберните в функцию `clean`

   * Немножко полистайте свои тексты, посмотрите проблемы, которые есть в них
   * Удалите ненужные сущности, например ссылки, смайлы, и т.д. Все, что считаете мусором и не нужным для кластеризации и только будет создавать лишние токены
  
💡 __совет__:
* Не пытайтесь сделать сразу идеальную отчистку, в разных текстах, разные проблемы, лучше сделать baseline-проходку, а дальше вернуться, уже видя проблемы и чдочищать их
* Можно сразу результат прикреплять в `DataFrame`, например создать колонку `cleaned_text`

In [62]:
def clean(text):
    text = text.lower()
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b[a-zA-Z]+\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

cleaned_text = []

for text in df['content']:
    cleaned_text.append(clean(text))

In [10]:
cleaned_text[110]

'рар выбрала новых академиков ими стали алексей поповичев исполнительный директор ассоциации русбренд роман фирайнер креативный директор агентства ольга барская вицепрезидент по развитию бизнеса академики выбрали событием года проект вкусно и точка а его лидером вицепрезидента по маркетингу дарью назаркинус года российская академия рекламы объединяет ведущих специалистов рекламного отечественного рынка членами академии становятся признанные эксперты отрасли которые вносят значительный вклад в развитие коммуникационной индустриирар содействует в формировании фундаментальных основ рекламной деятельности в стране разрабатывает индустриальные стандарты и критерии оценки качества рекламного продукта поддерживает молодых профессионалов и занимается развитием науки о коммуникациях в сфере рекламыподробнее рар'

__3.__ Удалите __стоп-слова__. Оберните в функцию `clean`

💡 __Небольшая подсказка__:
* В демо-файле с лекции мы подгружали список английских стоп-слов `stopwords.words('english')`, там есть и русский `stopwords.words('russian')`. Можете попробовать взять его, однако он содержит в себе не так много стоп-слов нашего языка
* [Здесь](https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt) добрые люди сделали более расширенный список стоп-слов, есть идея его подгрузить, дабы работало лучше

In [13]:
with open('/home/oldmovielover/Загрузки/stopwords-ru.txt', 'r', encoding='utf-8') as f:
    sw = set(f.read().splitlines())

In [66]:
def remove_stopwords(text, stop_words):
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

filtered_text = [remove_stopwords(word, sw) for word in cleaned_text]

In [21]:
filtered_text[110]

'рар выбрала новых академиков стали алексей поповичев исполнительный директор ассоциации русбренд роман фирайнер креативный директор агентства ольга барская вицепрезидент развитию бизнеса академики выбрали событием проект вкусно точка лидером вицепрезидента маркетингу дарью назаркинус российская академия рекламы объединяет ведущих специалистов рекламного отечественного рынка членами академии становятся признанные эксперты отрасли вносят значительный вклад развитие коммуникационной индустриирар содействует формировании фундаментальных основ рекламной деятельности стране разрабатывает индустриальные стандарты критерии оценки качества рекламного продукта поддерживает молодых профессионалов занимается развитием науки коммуникациях сфере рекламыподробнее рар'

__4.__ Проведите __лемматизацию__ или __стемминг__, что считаете нужным. Оберните в функцию `lemmitize`.

❗В демо-файле была работа над англоязычными текстами. Здесь текст русский, лемматизации в `nltk` для него нет.  
* Нам поможет [pymorphy3](https://pypi.org/project/pymorphy3/), работающий с русскими текстами.
      
* Ниже представлен небольшой пример. Можете попробовать попреобразовать разные слова и посмотреть результат, далее примените к своим текстам

In [52]:
morph = pymorphy3.MorphAnalyzer()  
morph.parse('апельсины')[0].normal_form, morph.parse('была')[0].normal_form

('апельсин', 'быть')

In [55]:
def lemmitize(text, morph):
    pass
    words = text.split()
    words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(words)

lemmitize_text = [lemmitize(word, morph) for word in filtered_text]

In [69]:
lemmitize_text[110]

'рар выбрать новый академик стать алексей поповичев исполнительный директор ассоциация русбренд роман фирайнер креативный директор агентство ольга барский вицепрезидент развитие бизнес академик выбрать событие проект вкусно точка лидер вицепрезидент маркетинг дарья назаркинус российский академия реклама объединять ведущий специалист рекламный отечественный рынок член академия становиться признанный эксперт отрасль вносить значительный вклад развитие коммуникационный индустриирар содействовать формирование фундаментальный основа рекламный деятельность страна разрабатывать индустриальный стандарт критерий оценка качество рекламный продукт поддерживать молодой профессионал заниматься развитие наука коммуникация сфера рекламыподробный рар'

__5.__ Векторизуйте тексты, любым способом `Tf-Idf`, `CountVetorizer`, [`HashingVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html). Отвекторизованный текст сохраните в `matrix_of_texts`

📑 Про более подробную работу `HashingVetorizer` можно почитать [тут](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html)

💡 __совет__:
* Создайте как датафрейм, матрицу(пример в демо), можно понять будет сколько токенов образовалось и каких

In [226]:
vectorizer = TfidfVectorizer(max_df=0.7, min_df=0.01)
X_tfidf = vectorizer.fit_transform(lemmitize_text)
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

n_samples: 2000, n_features: 662


In [227]:
matrix_of_texts = pd.DataFrame(data=X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
matrix_of_texts

,август,автор,агент,агентство,адрес,академия,актуальный,алгоритм,американский,анализ,...,элемент,этап,эфир,эффект,эффективный,являться,язык,январь,яндекс,яркий
0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.0,0.21143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.475401,0.0
4,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1996,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.228745,0.0,0.0,0.000000,0.000000,0.0
1997,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1998,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.487273,0.000000,0.0


__6.__ Понизьте размерность любым способом `TruncatedSVD`/`umap`. Сохраните результат в `reduced_matrix_of_texts`

In [260]:
svd = TruncatedSVD(n_components=15)
X_svd = svd.fit_transform(matrix_of_texts)
explained_variance = svd.explained_variance_ratio_.sum()

In [261]:
print(f"Explained variance of the SVD: {explained_variance * 100:.1f}%")
X_svd.shape

Explained variance of the SVD: 9.0%


(2000, 15)

__7.__ Проведи кластеризацию c помощью `K-means`, количество кластеров попробуйте сделать 5

In [262]:
# your_code
km = KMeans(n_clusters=5, random_state=42)
km.fit(X_svd)

KMeans(n_clusters=5, random_state=42)

__8.__ Выведи топ15 слов для каждого кластера

In [271]:
# your_code
original_space_centroids = svd.inverse_transform(km.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i, :15]:
        print(f"{terms[ind]} ", end="")
    print()

Cluster 0: врач организм пациент ребёнок урок университет совершенный лечение заболевание педиатрический помощь препарат мужчина работа здоровье 
Cluster 1: музей картина художник новый человек такой первый фильм история день один самый друг который искусство 
Cluster 2: канал пост интересный ссылка наш подписчик рекламный писать друг публикация самый чат очередной создать хороший 
Cluster 3: новый компания яндекс работа пользователь россия сайт проект приложение сервис работать дать данные использовать рынок 
Cluster 4: реклама рекламный бренд социальный канал рынок компания ролик новый кампания объём каждый официальный день рассказать 


__9.__ Посчитай внутренние(`silhouette_score`) и внешние метрики(`rand_score`). Для внешник метрик у вас есть разметка в исходных данных в столбце `label`

In [ ]:
from sklearn.metrics import silhouette_score, rand_score

In [269]:
print(f"Внутренние(`silhouette_score`: {silhouette_score(X_svd, km.labels_)})\nВнешние метрики(`rand_score`: {rand_score(label, km.labels_)})")

Внутренние(`silhouette_score`: 0.11077665724813454)
Внешние метрики(`rand_score`: 0.6606168084042021)


__10.__ Попробуйте построить `wordcloud` - ссылка на [туториал](https://www.datacamp.com/tutorial/wordcloud-python)

In [15]:
#your_code

__11.__ Задание со звездочкой. Оберни всю работу выше в итоговый пайплайн следующего вида:  
   * `MyCustomPreprocessorText` __->__ `CountVectorizer` __->__ `TruncatedSVD` __->__ `Kmeans`
   * `MyCustomPreprocessorText` внутри себя будет содержать все этапы предобработки и очистки текста:
        - `clean` __->__ `remove_stopwords` __->__ `lemmitize`
   * Обучи и подай на вход текст "Этот текст о рекламе, о маркетинге и его пользе для любого продукта для людей" и проверь, к какому кластеру отнесется данный объект

__Некоторый шаблон в помощи сборки своего препроцессора указан ниже__

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import pymorphy3

class MyCustomTextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = self.get_stopwords_list() 
        self.morph = pymorphy3.MorphAnalyzer()

    def fit(self, X, y=None): # Препроцессору обучаться не нужно, его нужно только преобразовывать, поэтому тут пусто
        return self

    def transform(self, texts, y=None): # Будет преобразоваывать
        return [self.preprocess(text) for text in texts]

    # Здесь можно прописать код получения словаря со стоп-словами
    def get_stopwords_list(self): 
        # your_code
        return # stop_words

    # Функция по очистке текста
    def clean(self, text):
        # вставьте сюда свою функцию clean из заданий выше
        return # text

    def remove_stopwords(self, text):
        # Здесь можно вставить свою функцию remove_stopwords написанную в заданиях выше
        return filtered_text
    
    
    def lemmatize(self, text):
        # Вставьте lemmatize
        return lemmatized_text

    # Функция, которая запустит последовательный запуск всех этапов выше
    def preprocess(self, text):
        text = self.clean(text)
        text = self.remove_stopwords(text)
        text = self.lemmatize(text)
        return text

* Проверьте его, работает ли правильно transform у этого класса, если все ок 

In [3]:
# pipeline = Pipeline([
#     ('preprocessor', MyCustomTextPreprocessor()),
# ])

# pipeline.transform(["Этот текст о рекламе, о маркетинге и его пользе для любого продукта для людей"])

* А теперь соберите полный пайплайн и посчитайте кластер данного текста